### Test of data instantiation

In [60]:
import torch
import numpy
from torch.utils.data import Dataset, DataLoader 
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
import pandas as pd
from PIL import Image
import os


In [61]:

        
# img = torch.ones(6, 3)
# labels = [0, 1, 1, 1, 0, 0]

# dataset = ExpDataSet(img, labels)
# print(dataset[4])



In [62]:
# Local paths

label_path = "dataset/datasets/rf/ringcodes.csv"

# Make sure to separate csv file on |
df = pd.read_csv(label_path, sep="|")

df.head(10)

,filename,code,color
0,"20240408-145102.064779(162.29,-4.57,9209.0).jpg",JE24X,NW
1,"20240408-145142.722154(103.96,-3.69,10150.0).jpg",JJ94C,NW
2,"20240408-145459.652977(162.36,-4.64,7886.0).jpg",JE24X,NW
3,"20240408-145544.814361(93.19,-3.36,10055.0).jpg",JJ94C,NW
4,"20240408-145602.037977(80.53,-3.07,10338.0).jpg",JE94C,NW
5,"20240408-152700.065700(-163.15,-3.84,10636.0).jpg",J194V,NW
6,"20240408-152816.726683(132.59,-1.38,10382.0).jpg",JJ89C,NW
7,"20240408-153023.711880(167.64,-5.87,8263.0).jpg",JE24X,NW
8,"20240408-153158.880806(100.81,-19.02,3832.0).jpg",J808V,NW
9,"20240408-154814(-173.23,-5.81,8512.0).jpg",JE25X,NW


In [63]:

dl = df.sort_values("filename", ascending=True).reset_index(drop=True)
dl.head(10)
# exp_labels = dl[["code", "color"]]
# exp_labels.head()


,filename,code,color
0,"20240408-145102.064779(162.29,-4.57,9209.0).jpg",JE24X,NW
1,"20240408-145142.722154(103.96,-3.69,10150.0).jpg",JJ94C,NW
2,"20240408-145459.652977(162.36,-4.64,7886.0).jpg",JE24X,NW
3,"20240408-145544.814361(93.19,-3.36,10055.0).jpg",JJ94C,NW
4,"20240408-145602.037977(80.53,-3.07,10338.0).jpg",JE94C,NW
5,"20240408-152700.065700(-163.15,-3.84,10636.0).jpg",J194V,NW
6,"20240408-152816.726683(132.59,-1.38,10382.0).jpg",JJ89C,NW
7,"20240408-153023.711880(167.64,-5.87,8263.0).jpg",JE24X,NW
8,"20240408-153158.880806(100.81,-19.02,3832.0).jpg",J808V,NW
9,"20240408-154814(-173.23,-5.81,8512.0).jpg",JE25X,NW


In [64]:
class ExpDataSet(Dataset):
    def __init__(self, img_path:str, labels_path:str, transform=None, max_n=None):
        self.img_path = img_path
        self.transform = transform
        self.max_n = max_n

        # Retrieve labels and sort them in alphabetical order to match the images
        df = pd.read_csv(labels_path, sep="|")
        df = df.sort_values("filename", ascending=True).reset_index(drop=True)

        # Ensure the data contains the chosen amount of elements
        if max_n is not None:
            df = df[:self.max_n]
        
        # Retrieve images from folder and match them with labels
        self.img_paths = df["filename"].apply(lambda e: os.path.join(img_path, e)).to_list()

        self.labels = list(df[["code","color"]].itertuples(index=False, name=None)) # Labels should be a list of tuples
        
    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        label = self.labels[idx]
        image = Image.open(self.img_paths[idx]).convert('RGB')

        if self.transform: # Add transformation mask to image
            image = self.transform(image)
        
        return image, label

In [106]:
image_path = "dataset/datasets/rf/images"
max_n = 10
transform = T.Compose([
    #T.Resize((224,224)),
    T.ToTensor()
])
exp_dataset = ExpDataSet(img_path=image_path, labels_path=label_path, transform=transform, max_n=max_n)


In [107]:

exp_loader = DataLoader(dataset=exp_dataset, batch_size=64, shuffle=True)

def tensor_to_numpy(img_tensor):
    img_np = img_tensor.detach().cpu()
    # img_np = img_np.permute(0, 2, 3, 1).numpy()
    img_np = img_np.permute(1, 2, 0).numpy()
    #img_np= (img_np* 255).clip(0, 255).astype('uint8')
    return img_np

In [72]:
from paddleocr import PaddleOCR as POCR

ocr = POCR(use_doc_orientation_classify=True, use_doc_unwarping=True, use_textline_orientation=True)


Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in C:\Users\legom\.paddlex\official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00,  8.98it/s]
Creating model: ('UVDoc', None)
Using official model (UVDoc), the model files will be automatically downloaded and saved in C:\Users\legom\.paddlex\official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 2053.18it/s]
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Using official model (PP-LCNet_x1_0_textline_ori), the model files will be automatically downloaded and saved in C:\Users\legom\.paddlex\official_models.
Creating model: ('PP-OCRv5_server_det', None)
Using official model (PP-OCRv5_server_det), the model files will be automatically downloaded and saved in C:\Users\legom\.paddlex\official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 2160.71it/s]
Creating model: ('PP-OCRv5_server_rec', No

In [ ]:
results = []
for images, labels in exp_loader:
    images = tensor_to_numpy(images)
    for image in images:
        result = ocr.predict(image)
        results.append(result)

In [ ]:
print(len(results))
for i in range(0, len(results)):
    #results[i].print()




10


AttributeError: 'list' object has no attribute 'print'

In [ ]:
# Sanity check
image = Image.open("other/sign.jpg").convert('RGB')
func = T.PILToTensor()
image = func(image)
image = tensor_to_numpy(image)
result = ocr.predict(image)
result[0].print()

{'res': {'input_path': None, 'page_index': None, 'model_settings': {'use_doc_preprocessor': True, 'use_textline_orientation': True}, 'doc_preprocessor_res': {'input_path': None, 'page_index': None, 'model_settings': {'use_doc_orientation_classify': True, 'use_doc_unwarping': True}, 'angle': 0}, 'dt_polys': array([[[1035,  103],
        ...,
        [1031,  188]],

       ...,

       [[  58,  192],
        ...,
        [  58,  351]]], shape=(3, 4, 2), dtype=int16), 'text_det_params': {'limit_side_len': 64, 'limit_type': 'min', 'thresh': 0.3, 'max_side_limit': 4000, 'box_thresh': 0.6, 'unclip_ratio': 1.5}, 'text_type': 'general', 'textline_orientation_angles': array([0, ..., 0], shape=(3,)), 'text_rec_score_thresh': 0.0, 'rec_texts': ['DR', 'Name', 'Street'], 'rec_scores': array([0.99681139, ..., 0.99957991], shape=(3,)), 'rec_polys': array([[[1035,  103],
        ...,
        [1031,  188]],

       ...,

       [[  58,  192],
        ...,
        [  58,  351]]], shape=(3, 4, 2), dtype=